# Deploy Web App on Azure Container Services (AKS)


In this notebook, we will set up an Azure Container Service which will be managed by Kubernetes. We will then take the Docker image we created earlier that contains our app and deploy it to the AKS cluster. Then, we will check everything is working by sending a question to it and getting it scored for matches in the original questions.

The process is split into the following steps:
- Define our resource names
- Login to Azure
- Create resource group and create AKS
- Connect to AKS
- Deploy our app

We assume that this notebook is running on Linux and Azure CLI is installed before proceeding.

In [1]:
import os
import json
import dotenv
from utilities import write_json_to_file
%load_ext dotenv

/anaconda/envs/MLAKSDeployment/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Setup

Determine the location of the dotenv file.

In [2]:
dotenv_path = dotenv.find_dotenv()

Below are the various name definitions for the resources needed to setup AKS. Please modify the values in the `set_key` statements as you see fit.

In [3]:
# If you have multiple subscriptions select the subscription you want to use 
dotenv.set_key(dotenv_path, 'selected_subscription', 'Microsoft Azure')

# Resource group, name and location for AKS cluster
dotenv.set_key(dotenv_path, 'resource_group', 'exampleAKS') 
dotenv.set_key(dotenv_path, 'aks_name', 'exampleAKS')
dotenv.set_key(dotenv_path, 'location', 'southeastasia')

(True, 'location', 'southeastasia')

Import the dotenv variables. Use them to determine the name of the image.

In [4]:
%dotenv -o
image_name = os.getenv('docker_login') + os.getenv('image_repo')
image_name

'kwarodom/mlaksdep'

In [5]:
os.getenv('selected_subscription')

'Microsoft Azure'

## Azure account login

The commands below will ensure you are logged in to your Azure account. If you are not logged in, it will initiate a login by popping up a browser at a login page where you will select your Azure account.

In [38]:
%%bash
list=`az account list -o table`
if [ "$list" == '[]' ] || [ "$list" == '' ]; then 
  az login -o table
else
  az account list -o table 
fi

Name             CloudName    SubscriptionId                        State    IsDefault
---------------  -----------  ------------------------------------  -------  -----------
Microsoft Azure  AzureCloud   f973e87c-e4b6-44f1-95ea-2f146d41c844  Enabled  True


In [39]:
!sudo az account set --subscription "$selected_subscription"

In [40]:
!az account show

{
  "environmentName": "AzureCloud",
  "id": "f973e87c-e4b6-44f1-95ea-2f146d41c844",
  "isDefault": true,
  "name": "Microsoft Azure",
  "state": "Enabled",
  "tenantId": "7312bb0d-08fa-4d15-92e6-df809bfdbf2f",
  "user": {
    "name": "admin@smarthomepea.onmicrosoft.com",
    "type": "user"
  }
}


You will also need to register the container service resources on your subscription if you haven't already done so.

In [41]:
!sudo az provider register -n Microsoft.ContainerService

Registering is still on-going. You can monitor using 'az provider show -n Microsoft.ContainerService'


In [42]:
!sudo az provider show -n Microsoft.ContainerService

{
  "authorization": {
    "applicationId": "7319c514-987d-4e9b-ac3d-d38c4f427f4c",
    "managedByRoleDefinitionId": "8e3af657-a8ff-443c-a75c-2fe8c4bcb635",
    "roleDefinitionId": "1b4a0c7f-2217-416f-acfa-cf73452fdc1c"
  },
  "id": "/subscriptions/f973e87c-e4b6-44f1-95ea-2f146d41c844/providers/Microsoft.ContainerService",
  "namespace": "Microsoft.ContainerService",
  "registrationState": "Registered",
  "resourceTypes": [
    {
      "aliases": null,
      "apiVersions": [
        "2017-07-01",
        "2017-01-31",
        "2016-09-30",
        "2016-03-30"
      ],
      "capabilities": "None",
      "locations": [
        "Japan East",
        "Central US",
        "East US 2",
        "Japan West",
        "East Asia",
        "South Central US",
        "Australia East",
        "Australia Southeast",
        "Brazil South",
        "Southeast Asia",
        "West US",
        "North Central US",
        "West Europe",
        "North Europe",
        "East US",
        "UK West"

## Create resources and dependencies

### Create resource group and AKS cluster

Azure encourages the use of groups to organize all the Azure components you deploy. That way it is easier to find them but also we can delete a number of resources simply by deleting the group.

In [43]:
!sudo az group create --name $resource_group --location $location

{
  "id": "/subscriptions/f973e87c-e4b6-44f1-95ea-2f146d41c844/resourceGroups/exampleAKS",
  "location": "southeastasia",
  "managedBy": null,
  "name": "exampleAKS",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null
}


Below, we create the AKS cluster  with 5 nodes in the resource group we created earlier. This step can take ten or more minutes.

In [ ]:
%%time
!sudo az aks create --resource-group $resource_group --name $aks_name --node-count 5 --generate-ssh-keys -s Standard_D4_v2

### Install kubectl CLI

To connect to the Kubernetes cluster, we will use kubectl, the Kubernetes command-line client. To install, run the following:

In [6]:
!sudo env "PATH=$PATH" az aks install-cli

Please ensure that /usr/local/bin is in your search PATH, so the `kubectl` command can be found.


## Connect to AKS cluster

To configure kubectl to connect to the Kubernetes cluster, run the following command:

In [8]:
!sudo az aks get-credentials --resource-group $resource_group --name $aks_name

Merged "exampleAKS" as current context in /home/kwarodom/.kube/config


Let's verify connection by listing the nodes.

In [10]:
!sudo kubectl get nodes

NAME                       STATUS   ROLES   AGE     VERSION
aks-nodepool1-36721700-0   Ready    agent   6m45s   v1.9.11
aks-nodepool1-36721700-1   Ready    agent   7m1s    v1.9.11
aks-nodepool1-36721700-2   Ready    agent   6m45s   v1.9.11
aks-nodepool1-36721700-3   Ready    agent   6m43s   v1.9.11
aks-nodepool1-36721700-4   Ready    agent   6m31s   v1.9.11


Let's check the pods on our cluster.

In [12]:
!sudo kubectl get pods --all-namespaces

NAMESPACE     NAME                                   READY   STATUS    RESTARTS   AGE
kube-system   heapster-5884fdbc48-fc9d8              2/2     Running   0          20m
kube-system   kube-dns-v20-b8ff799f7-5b6xq           3/3     Running   0          20m
kube-system   kube-dns-v20-b8ff799f7-8hh8m           3/3     Running   0          20m
kube-system   kube-proxy-9qbzx                       1/1     Running   0          12m
kube-system   kube-proxy-cgjdh                       1/1     Running   0          12m
kube-system   kube-proxy-dhpfb                       1/1     Running   0          12m
kube-system   kube-proxy-ghgqs                       1/1     Running   0          12m
kube-system   kube-proxy-md6gg                       1/1     Running   0          12m
kube-system   kube-svc-redirect-5xn9g                2/2     Running   0          12m
kube-system   kube-svc-redirect-jdp49                2/2     Running   0          12m
kube-system   kube-svc-redirect-jf5sb                2

## Deploy application

Below we define our Kubernetes manifest file for our service and load balancer. Note that we have to specify the image name and cpu requests and limits for pods. We first start with  deploying 2 pods.

In [13]:
app_template = {
  "apiVersion": "apps/v1beta1",
  "kind": "Deployment",
  "metadata": {
      "name": "azure-ml"
  },
  "spec":{
      "replicas":2,
      "template":{
          "metadata":{
              "labels":{
                  "app":"azure-ml"
              }
          },
          "spec":{
              "containers":[
                  {
                      "name": "azure-ml",
                      "image": image_name,

                      "ports":[
                          {
                              "containerPort":80,
                              "name":"model"
                          }
                      ],
                      "resources":{
                           "requests":{
                               "cpu": 1
                           },
                           "limits":{
                               "cpu": 1.25
                           }
                       }  
                  }
              ]
          }
      }
  }
}

service_temp = {
  "apiVersion": "v1",
  "kind": "Service",
  "metadata": {
      "name": "azure-ml"
  },
  "spec":{
      "type": "LoadBalancer",
      "ports":[
          {
              "port":80
          }
      ],
      "selector":{
            "app":"azure-ml"
      }
   }
}

In [14]:
write_json_to_file(app_template, 'az-ml.json')

In [15]:
write_json_to_file(service_temp, 'az-ml.json', mode='a')

Let's check the manifest created.

In [16]:
!cat az-ml.json

{
    "apiVersion": "apps/v1beta1",
    "kind": "Deployment",
    "metadata": {
        "name": "azure-ml"
    },
    "spec": {
        "replicas": 2,
        "template": {
            "metadata": {
                "labels": {
                    "app": "azure-ml"
                }
            },
            "spec": {
                "containers": [
                    {
                        "image": "kwarodom/mlaksdep",
                        "name": "azure-ml",
                        "ports": [
                            {
                                "containerPort": 80,
                                "name": "model"
                            }
                        ],
                        "resources": {
                            "limits": {
                                "cpu": 1.25
                            },
                            "requests": {
                                "cpu": 1
                            }
                        }
            

Next, we will use kubectl create command to deploy our application.

In [18]:
!sudo kubectl create -f az-ml.json

deployment.apps/azure-ml created
service/azure-ml created


Let's check if the pod is deployed. It can take several minutes for the deployment to be ready and running.

In [19]:
!sudo kubectl get pods --all-namespaces

NAMESPACE     NAME                                   READY   STATUS              RESTARTS   AGE
default       azure-ml-b5f8f9478-btsj6               0/1     ContainerCreating   0          6s
default       azure-ml-b5f8f9478-lp5cp               0/1     ContainerCreating   0          6s
kube-system   heapster-5884fdbc48-fc9d8              2/2     Running             0          25m
kube-system   kube-dns-v20-b8ff799f7-5b6xq           3/3     Running             0          25m
kube-system   kube-dns-v20-b8ff799f7-8hh8m           3/3     Running             0          25m
kube-system   kube-proxy-9qbzx                       1/1     Running             0          17m
kube-system   kube-proxy-cgjdh                       1/1     Running             0          17m
kube-system   kube-proxy-dhpfb                       1/1     Running             0          17m
kube-system   kube-proxy-ghgqs                       1/1     Running             0          17m
kube-system   kube-proxy-md6gg            

If anything goes wrong you can use the commands below to observe the events on the node as well as review the logs.

In [20]:
!sudo kubectl get events

LAST SEEN   TYPE      REASON                       KIND         MESSAGE
22m         Normal    NodeHasSufficientDisk        Node         Node aks-nodepool1-36721700-0 status is now: NodeHasSufficientDisk
22m         Normal    NodeHasSufficientMemory      Node         Node aks-nodepool1-36721700-0 status is now: NodeHasSufficientMemory
22m         Normal    NodeHasNoDiskPressure        Node         Node aks-nodepool1-36721700-0 status is now: NodeHasNoDiskPressure
22m         Normal    NodeAllocatableEnforced      Node         Updated Node Allocatable limit across pods
17m         Normal    RegisteredNode               Node         Node aks-nodepool1-36721700-0 event: Registered Node aks-nodepool1-36721700-0 in Controller
17m         Normal    Starting                     Node         Starting kube-proxy.
22m         Normal    NodeHasSufficientDisk        Node         Node aks-nodepool1-36721700-1 status is now: NodeHasSufficientDisk
22m         Normal    NodeHasSufficientMemory      Nod

Check the logs for the first application pod.

In [22]:
pod_json = !sudo kubectl get pods -o json
pod_dict = json.loads(''.join(pod_json))

In [23]:
!sudo kubectl logs {pod_dict['items'][0]['metadata']['name']}

Error from server (BadRequest): container "azure-ml" in pod "azure-ml-b5f8f9478-btsj6" is waiting to start: ContainerCreating


In [ ]:
!kubectl get deployment

It can take a few minutes for the service to populate the EXTERNAL-IP field below. This will be the IP you use to call the service. You can also specify an IP to use, please see the AKS documentation for further details.

In [ ]:
!kubectl get service azure-ml

# Scaling

In this part, we scale the number of pods to make sure we fully utilize the AKS cluster.

In [ ]:
!kubectl scale --current-replicas=2 --replicas=35 deployment/azure-ml

It can take a couple of minutes for all replicas to be running.

In [ ]:
!kubectl get pods --all-namespaces

In [ ]:
!kubectl get deployment

Next, we will [test our web application deployed on AKS](06_Test_WebApp.ipynb).

Once, you are done with all the notebooks of the tutorial, you can use the instructions in the [last notebook](09_Tear_Down.ipynb) to tear down the cluster.